In [52]:
import os
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch3d
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes, knn_points, estimate_pointcloud_normals, knn_gather, cubify

import trimesh

from torch_scatter import scatter

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')

import torch
import pytorch3d 
from pytorch3d.ops import knn_points,knn_gather
from pytorch3d.transforms import axis_angle_to_matrix, matrix_to_axis_angle


from tqdm import tqdm

from pytorch3d.utils import ico_sphere

# # from pytorch3d.structures import Meshes

# from .utils import one_hot_sparse


In [53]:
from ops.mesh_geometry import *
from ops.pcl_geometry import *

In [62]:
device = torch.device("cuda:0")


trg_mesh = load_objs_as_meshes(["kitty.obj"], device=device)

# trg_mesh = ico_sphere(5, device)



trg_mesh = normalize_mesh(trg_mesh)

/home/yihao/anaconda3/envs/geometery3d/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [63]:

voxelizer = Differentiable_Voxelizer(bbox_density=128)
voxel_trg = voxelizer.forward(trg_mesh, output_resolution=128)


# cubified = cubify(voxel_trg, 0.5)

In [64]:
inside_points = torch.where(voxel_trg[0] > 0.5)
outside_points = torch.where(voxel_trg[0] < 0.5)

window_size = voxel_trg.shape[1:]
inside_points = torch.stack([inside_points[2]*2/(window_size[2]-1)-1, inside_points[1]*2/(window_size[1]-1)-1, inside_points[0]*2/(window_size[0]-1)-1], dim=-1)
outside_points = torch.stack([outside_points[2]*2/(window_size[2]-1)-1, outside_points[1]*2/(window_size[1]-1)-1, outside_points[0]*2/(window_size[0]-1)-1], dim=-1)


In [65]:
num_samples = 100000
pointscloud, normals_gt = sample_points_from_meshes(trg_mesh, num_samples, return_normals=True)

In [66]:
pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 10)


curvature = pcl_with_frames.gaussian_curvature().view(-1)

area = pcl_with_frames.local_voronoi_area().view(-1)



In [67]:
for i in range(3,9):
    pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 10*i)
    print('Estimated Genus',  1-torch.round(pcl_with_frames.differentiable_euler_number()).item()/2)

    print('Estimated euler',  pcl_with_frames.differentiable_euler_number())

# parametric stability

Estimated Genus 1.0
Estimated euler tensor(0.1277, device='cuda:0')
Estimated Genus 1.0
Estimated euler tensor(0.1443, device='cuda:0')
Estimated Genus 1.0
Estimated euler tensor(0.1536, device='cuda:0')
Estimated Genus 1.0
Estimated euler tensor(0.1915, device='cuda:0')
Estimated Genus 1.0
Estimated euler tensor(0.2114, device='cuda:0')
Estimated Genus 1.0
Estimated euler tensor(0.2747, device='cuda:0')


In [68]:
pl = pv.Plotter(notebook=True)
rescaled_curv = torch.tanh(curvature*area*200)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[0], scalars=rescaled_curv.detach().cpu().numpy(), point_size=5,cmap="viridis")

pl.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [69]:
## differentiable optimization 


# frames_field_axis_angle = matrix_to_axis_angle(pcl_with_frames.frames_field)
# frames_field_axis_angle.requires_grad = True

# optimizer = torch.optim.AdamW([frames_field_axis_angle], lr=1e-1)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.5)


# loop = tqdm(range(1000))

# for epoch in loop:

#     optimizer.zero_grad()

#     frames_field = axis_angle_to_matrix(frames_field_axis_angle)

#     normals_field = frames_field[:,:,0:1,:]
#     tangent1_field = frames_field[:,:,1:2,:]
#     tangent2_field = frames_field[:,:,2:3,:]

#     local_pt_difference = pcl_with_frames.knn_info.knn - pcl_with_frames.pointscloud[:,:,None,:] # B x N x K x 3

#     voronoi_area_list = pcl_with_frames.local_voronoi_area(frames_field)

#     # euler = pcl_with_frames.differentiable_euler_number(frames_field)



#     # # project the difference onto the tangent plane, getting the differential of the gaussian map
#     # local_dpt_tangent1 = (local_pt_difference * tangent1_field).sum(-1,keepdim=True)
#     # local_dpt_tangent2 = (local_pt_difference * tangent2_field).sum(-1,keepdim=True)


#     Loss_perpendicular = (local_pt_difference * normals_field).sum(-1,keepdim=True)

#     Loss_perpendicular = F.mse_loss(Loss_perpendicular, torch.zeros_like(Loss_perpendicular))



#     patch_size = 2000

    

    
#     try:
#         winding_number_field_inside = (Electric_strength(inside_points_sampled, pcl_with_frames.pointscloud.view(-1,3))*normals_field[:,:,0,:]).sum(-1,keepdim=True)

#         winding_number_field_inside = (winding_number_field_inside*voronoi_area_list).sum(-2)/np.pi/4
    
#     except:
#         winding_number_field_inside = torch.zeros_like(inside_points_sampled[:,0:1])
#         for i in range(pcl_with_frames.pointscloud.shape[1]//patch_size+1):
#             winding_number_field_inside_temp = (Electric_strength(inside_points_sampled, pcl_with_frames.pointscloud[:,i*patch_size:(i+1)*patch_size].view(-1,3))*frames_field[:,i*patch_size:(i+1)*patch_size,0,:]).sum(-1,keepdim=True)
#             winding_number_field_inside_temp = (winding_number_field_inside_temp*voronoi_area_list[:,i*patch_size:(i+1)*patch_size]).sum(-2)/np.pi/4

#             winding_number_field_inside += winding_number_field_inside_temp

    
    
#     try:
#         winding_number_field_outside = (Electric_strength(outside_points_sampled, pcl_with_frames.pointscloud.view(-1,3))*normals_field[:,:,0,:]).sum(-1,keepdim=True)

#         winding_number_field_outside = (winding_number_field_outside*voronoi_area_list).sum(-2)/np.pi/4
#     except:
#         winding_number_field_outside = torch.zeros_like(outside_points_sampled[:,0:1])
#         for i in range(pcl_with_frames.pointscloud.shape[1]//patch_size+1):
#             winding_number_field_outside_temp = (Electric_strength(outside_points_sampled, pcl_with_frames.pointscloud[:,i*patch_size:(i+1)*patch_size].view(-1,3))*frames_field[:,i*patch_size:(i+1)*patch_size,0,:]).sum(-1,keepdim=True)
#             winding_number_field_outside_temp = (winding_number_field_outside_temp*voronoi_area_list[:,i*patch_size:(i+1)*patch_size]).sum(-2)/np.pi/4

#             winding_number_field_outside += winding_number_field_outside_temp



#     Loss_occupancy = F.mse_loss(winding_number_field_inside, torch.ones_like(winding_number_field_inside)) + F.mse_loss(winding_number_field_outside, torch.zeros_like(winding_number_field_outside))






#     local_normals_difference = knn_gather(normals_field.squeeze(-2),pcl_with_frames.knn_info.idx) - normals_field
#     # project the difference onto the tangent plane, getting the differential of the gaussian map

#     # Loss_normal_consistency = F.mse_loss(knn_gather(normals_field,knn_info.idx) - knn_gather(normals_field,knn_info.idx).mean(-2,keepdim=True), torch.zeros_like(knn_gather(normals_field,knn_info.idx)))


#     Loss_perpendicular_normals = (local_normals_difference * normals_field).sum(-1,keepdim=True)
#     Loss_perpendicular_normals = F.mse_loss(Loss_perpendicular_normals, torch.zeros_like(Loss_perpendicular_normals))

 

#     # Loss_euler = torch.sin(np.pi*euler-np.pi/2)+1 # a closed orientatiable manifold has even euler number


#     Loss = Loss_occupancy  
#     # +  1e-6*Loss_euler + 0.05*Loss_normal_consistency + 0.1*Loss_perpendicular_normals + 0.1*Loss_tangent_dnormal


#     if epoch%20  == 0:
#         eulers = pcl_with_frames.differentiable_euler_number(frames_field)

#     loop.set_description('Loss: {:.4f}'.format(Loss.item())+', area: {:.4f}'.format(voronoi_area_list.sum().item())+', euler: {:.4f}'.format(eulers.sum().item()))

    
#     Loss.backward()
#     optimizer.step()
#     scheduler.step()



    

In [ ]:
# pointscloud_np = pointscloud[0].detach().cpu().numpy()

# # visualize the moving frame 
# pl = pv.Plotter(notebook=True)
# # pl.add_points(trg_mesh.verts_packed().detach().cpu().numpy(), color='r', opacity = 0)

# # pl.add_points(inside_points.detach().cpu().numpy(), color='g', opacity = 0.2)
# pl.add_mesh(trg_trimesh, color='lightgreen', opacity=0.2)


# pl.add_arrows(pointscloud_np, frames_field[0,:,0,:].detach().cpu().numpy(), color='b', opacity=0.2, mag=0.05)


# pl.show()